In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
from scipy.stats import f_oneway

# Set the style for seaborn
sns.set(style="whitegrid")

In [27]:
 #Load the data
data = pd.read_csv('../data/MachineLearningRating_v3.csv')

# Display the first few rows of the dataframe
data.head()

/tmp/ipykernel_10153/3149716167.py:2: DtypeWarning: Columns (32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../data/MachineLearningRating_v3.csv')


,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,...,ExcessSelected,CoverCategory,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims
0,145249,12827,2015-03-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
1,145249,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
2,145249,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0
3,145255,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,0.0
4,145255,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0


In [34]:
data.Province.value_counts()

Province
Gauteng          393865
Western Cape     170796
KwaZulu-Natal    169781
North West       143287
Mpumalanga        52718
Eastern Cape      30336
Limpopo           24836
Free State         8099
Northern Cape      6380
Name: count, dtype: int64

In [24]:
from scipy.stats import ttest_ind, chi2_contingency

# Group A: Plans in Province A
group_a = data[data['Province'] == 'Province']['TotalClaims']

# Group B: Plans in Province B
group_b = data[data['Province'] == 'Province']['TotalClaims']

# Perform t-test
t_stat, p_value = ttest_ind(group_a, group_b, equal_var=False)

print(f"T-statistic: {t_stat}, P-value: {p_value}")

if p_value < 0.05:
    print("Reject the null hypothesis: There are risk differences across provinces.")
else:
    print("Fail to reject the null hypothesis: There are no risk differences across provinces.")

T-statistic: nan, P-value: nan
Fail to reject the null hypothesis: There are no risk differences across provinces.


/tmp/ipykernel_10153/2514817195.py:10: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  t_stat, p_value = ttest_ind(group_a, group_b, equal_var=False)


In [40]:
import pandas as pd
from scipy.stats import ttest_ind


# Filtering the dataset by Province for two specific provinces
gauteng_claims = data[data['Province'] == 'Gauteng']['TotalClaims']
Western_Cape_claims = data[data['Province'] == 'Western Cape']['TotalClaims']

# Check group sizes and handle NaN
print(f"Gauteng size before dropna: {len(gauteng_claims)}, Western Cape size before dropna: {len(Western_Cape_claims)}")
gauteng_claims = gauteng_claims.dropna()
Western_Cape_claims = Western_Cape_claims.dropna()
print(f"Gauteng size after dropna: {len(gauteng_claims)}, Western Cape size after dropna: {len(Western_Cape_claims)}")

# Perform t-test if groups are valid
if len(gauteng_claims) == 0 or len(Western_Cape_claims) == 0:
    print("One of the groups is empty. Check your filtering conditions.")
else:
    t_stat, p_value = ttest_ind(gauteng_claims, Western_Cape_claims, equal_var=False)
    print(f"T-statistic: {t_stat}, P-value: {p_value}")

    # Analyze the p-value
    if p_value < 0.05:
        print("Reject the null hypothesis: There are risk differences across provinces.")
    else:
        print("Fail to reject the null hypothesis: There are no risk differences across provinces.")


Gauteng size before dropna: 393865, Western Cape size before dropna: 170796
Gauteng size after dropna: 393865, Western Cape size after dropna: 170796
T-statistic: 1.8652143496485993, P-value: 0.062152314522800046
Fail to reject the null hypothesis: There are no risk differences across provinces.


In [42]:
# Ensure no extra spaces and correct filtering
group_a = data[data['Province'].str.strip() == 'Gauteng']['TotalClaims']
group_b = data[data['Province'].str.strip() == 'KwaZulu-Natal']['TotalClaims']

# Check group sizes and handle NaN
print(f"Gauteng size before dropna: {len(group_a)}, KwaZulu-Natal size before dropna: {len(group_b)}")

# Drop NaN values in TotalClaims for both groups
group_a = group_a.dropna()
group_b = group_b.dropna()

print(f"Gauteng size after dropna: {len(group_a)}, KwaZulu-Natal size after dropna: {len(group_b)}")

# Check if either group is empty
if len(group_a) == 0:
    print("Group A (Gauteng) is empty. Check your filtering conditions or data.")
elif len(group_b) == 0:
    print("Group B (KwaZulu-Natal) is empty. Check your filtering conditions or data.")
else:
    # Perform t-test if both groups have data
    t_stat, p_value = ttest_ind(group_a, group_b, equal_var=False)
    print(f"T-statistic: {t_stat}, P-value: {p_value}")

    # Interpret the results of the t-test
    if p_value < 0.05:
        print("Reject the null hypothesis: There are risk differences across provinces.")
    else:
        print("Fail to reject the null hypothesis: There are no risk differences across provinces.")


Gauteng size before dropna: 393865, KwaZulu-Natal size before dropna: 169781
Gauteng size after dropna: 393865, KwaZulu-Natal size after dropna: 169781
T-statistic: -1.246201702257548, P-value: 0.2126913533358068
Fail to reject the null hypothesis: There are no risk differences across provinces.


In [38]:
#Perform ANOVA to compare risk (TotalClaims) across multiple provinces. 
from scipy.stats import f_oneway

# List of provinces
provinces = data['Province'].unique()

# Group data by province
groups = [data[data['Province'] == province]['TotalClaims'].dropna() for province in provinces]

# Perform ANOVA
f_stat, p_value = f_oneway(*groups)

print(f"F-statistic: {f_stat}, P-value: {p_value}")

if p_value < 0.05:
    print("Reject the null hypothesis: There are risk differences across provinces.")
else:
    print("Fail to reject the null hypothesis: There are no risk differences across provinces.")


F-statistic: 5.849413762407606, P-value: 1.6782057588675906e-07
Reject the null hypothesis: There are risk differences across provinces.


In [15]:
# List of zip codes
zip_codes = data['PostalCode'].unique()

# Group data by zip code
groups_zip = [data[data['PostalCode'] == zip_code]['TotalClaims'].dropna() for zip_code in zip_codes]

# Perform ANOVA
f_stat, p_value = f_oneway(*groups_zip)

print(f"F-statistic: {f_stat}, P-value: {p_value}")

if p_value < 0.05:
    print("Reject the null hypothesis: There are risk differences between zip codes.")
else:
    print("Fail to reject the null hypothesis: There are no risk differences between zip codes.")


F-statistic: 0.9419762214391849, P-value: 0.8906511279164051
Fail to reject the null hypothesis: There are no risk differences between zip codes.


In [21]:
# Group data by zip code and profit margin
groups_profit = [data[data['PostalCode'] == zip_code]['CalculatedPremiumPerTerm'].dropna() for zip_code in zip_codes]

# Perform ANOVA
f_stat, p_value = f_oneway(*groups_profit)

print(f"F-statistic: {f_stat}, P-value: {p_value}")

if p_value < 0.05:
    print("Reject the null hypothesis: There are significant profit differences between zip codes.")
else:
    print("Fail to reject the null hypothesis: There are no significant profit differences between zip codes.")



F-statistic: 5.715375005231982, P-value: 0.0
Reject the null hypothesis: There are significant profit differences between zip codes.


In [20]:
# Group data by Gender
group_women = data[data['Gender'] == 'Female']['TotalClaims'].dropna()
group_men = data[data['Gender'] == 'Male']['TotalClaims'].dropna()

# Perform t-test
t_stat, p_value = ttest_ind(group_women, group_men, equal_var=False)

print(f"T-statistic: {t_stat}, P-value: {p_value}")

if p_value < 0.05:
    print("Reject the null hypothesis: There are significant risk differences between Women and Men.")
else:
    print("Fail to reject the null hypothesis: There are no significant risk differences between Women and Men.")


T-statistic: 0.296353891400699, P-value: 0.7669656471629475
Fail to reject the null hypothesis: There are no significant risk differences between Women and Men.
